<a href="https://colab.research.google.com/github/Denev6/practice/blob/main/transformer/t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

import nltk
nltk.download("punkt")

Make sure to restart runtime after installing `sentencepiece`

In [ ]:
!python --version

Python 3.7.15


In [ ]:
import os
from google.colab import drive

import torch
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split 

In [ ]:
drive.mount("/content/drive")
CWD = "/content/drive/MyDrive/T5"

def join_path(*args):
    return os.path.join(CWD, *args)

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
TRAIN_CSV = join_path("data", "train.csv")
TEST_CSV = join_path("data", "test.csv")
MODEL = "t5-base"
MODEL_DIR = "t5-base"
BATCH_SIZE = 32
EPOCHS = 30
MAX_LENGTH = 128

TRAIN_ARGS = TrainingArguments(
    output_dir=join_path(MODEL_DIR),
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,  
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=7e-6,
    warmup_steps=500,
    weight_decay=1e-5,
    dataloader_num_workers=0,
    save_total_limit=1,
    save_strategy="no",
    evaluation_strategy="epoch"
)

Mounted at /content/drive


In [ ]:
train_csv = pd.read_csv(TRAIN_CSV)
train_csv.head()

In [ ]:
targets = train_csv["Target"].unique()
target_size = targets.shape[0]
target_size

7

In [ ]:
train_csv = train_csv.loc[:, ["Utterance", "Target"]]

df_train, df_eval = train_test_split(train_csv, test_size=0.2)
df_train.reset_index(drop=True, inplace=True)
df_eval.reset_index(drop=True, inplace=True)

In [ ]:
class EmotionDataset(Dataset):
    def __init__(self, df, tokenizer, target_size=7, mode=None, max_len=512):
        self.data_col = "Utterance"
        self.target_col = "Target"
        self.mode = str(mode).strip().lower()
        self.tokenizer = tokenizer
        self.target_size = target_size
        self.max_len = max_len
        self.data = self._tokenize(df)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        data, target = self.data[index]
        data_ids = data["input_ids"].squeeze()
        data_mask = data["attention_mask"].squeeze()

        if self.mode != "test":
            target_mask = target["attention_mask"].squeeze()
            labels = target["input_ids"].squeeze()
            labels[labels[:] == self.tokenizer.pad_token_id] = -100
            return {
                "input_ids": data_ids, 
                "attention_mask": data_mask, 
                "decoder_attention_mask": target_mask, 
                "labels": labels
            }

        return {
            "input_ids": data_ids, 
            "attention_mask": data_mask
        }

    def _tokenize(self, df):
        res = list()
        for index in range(df.shape[0]):
            if self.mode == "test":
                tokenized_target = None
            else:
                target = df.loc[index, self.target_col]
                tokenized_target = self.tokenizer(
                    [target],
                    max_length=self.target_size, 
                    padding="max_length", 
                    truncation=True, 
                    return_tensors="pt"
                )
            
            data = df.loc[index, self.data_col]
            tokenized_data = self.tokenizer(
                [data], 
                max_length=self.max_len, 
                padding="max_length", 
                truncation=True, 
                return_tensors="pt"
            )
            res.append([tokenized_data, tokenized_target])
        return res 

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(
    MODEL,
    max_length=MAX_LENGTH
)

In [ ]:
train_set = EmotionDataset(
    df_train, 
    tokenizer, 
    target_size=target_size, 
    mode="train", 
    max_len=MAX_LENGTH
)
eval_set = EmotionDataset(
    df_eval, 
    tokenizer, 
    target_size=target_size, 
    mode="train", 
    max_len=MAX_LENGTH
)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(
    MODEL,
    num_labels=target_size,
    ignore_mismatched_sizes=True
)

In [ ]:
def eval(model, tokenizer, eval_set):
    model.to(DEVICE)
    pred_values = list()
    
    for data in eval_set:
        output = model.generate(
            input_ids=data["input_ids"].unsqueeze(0).to(DEVICE), 
            attention_mask=data["attention_mask"].unsqueeze(0).to(DEVICE), 
            max_length=512
        )
        pred = [tokenizer.decode(ids) for ids in output][0]
        re_tag = re.compile("<.*?>")
        pred = re.sub(re_tag, "", pred).strip()
        pred_values.append(pred)

    real_values = df_eval["Target"].tolist()
    acc = accuracy_score(real_values, pred_values)
    f1 = f1_score(real_values, pred_values, average="macro")
    return {
        "accuracy": acc,
        "f1": f1
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=TRAIN_ARGS,
    train_dataset=train_set,
    eval_dataset=eval_set
)
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(join_path(MODEL_DIR))
torch.cuda.empty_cache()

In [ ]:
score = eval(model, tokenizer, eval_set)
print(f"Accuracy: {score['accuracy']:.5f}")
print(f"F1-macro: {score['f1']:.5f}")

In [ ]:
def classify(model, test_set):
    model.to(DEVICE)
    pred_values = list()
    
    for data in test_set:
        output = model.generate(
            input_ids=data["input_ids"].unsqueeze(0).to(DEVICE), 
            attention_mask=data["attention_mask"].unsqueeze(0).to(DEVICE), 
            max_length=512
        )
        pred = [tokenizer.decode(ids) for ids in output][0]
        re_tag = re.compile("<.*?>")
        pred = re.sub(re_tag, "", pred).strip()
        pred_values.append(pred)
    
    return pred_values

In [ ]:
test_csv = pd.read_csv(join_path("data", "test.csv"))
df_test = test_csv.loc[:, "Utterance"].to_frame()
test_set = EmotionDataset(df_test, tokenizer)

preds = classify(model, test_set)
test_csv["Target"] = preds

result_csv = test_csv.loc[:, ["ID", "Target"]]
result_csv.head()